In [1]:
import cv2
import os
import numpy as np
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input, Dense, Activation, Reshape, BatchNormalization, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer, Dense, Reshape, Lambda

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
train_path = '/kaggle/input/garbage-classification-v2/Garbage Classification'

def load_dataset(path, target_shape):
    class_folders = os.listdir(path)
    filenames = []
    labels = []

    for i, class_folder in enumerate(class_folders):
        folder_path = os.path.join(path, class_folder)
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            try:
                image = cv2.imread(image_path)
                if image is None:
                    continue  # Skip non-image files
                resized_image = cv2.resize(image, target_shape[:2])
                filenames.append(resized_image)
                labels.append(i)
            except Exception as e:
                print(f"Error processing image: {image_path} - {e}")

    X = np.array(filenames)
    y = np.array(labels)

    # Normalize pixel values
    X = X.astype('float32') / 255.0

    # Convert labels to categorical
    y = np_utils.to_categorical(y)

    return X, y


# Define the target shape for resizing images
target_shape = (110, 110, 3)

# Load and preprocess the dataset
X, y = load_dataset(train_path, target_shape)

# Split the dataset into training, testing, and validation sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Shape of X_train, y_train:", X_train.shape, y_train.shape)
print("Shape of X_test, y_test:", X_test.shape, y_test.shape)
print("Shape of X_val, y_val:", X_val.shape, y_val.shape)

    

libpng warning: Out of place sRGB chunk
libpng warning: Out of place sRGB chunk
libpng warning: Out of place sRGB chunk
libpng warning: Out of place sRGB chunk


Shape of X_train, y_train: (15827, 110, 110, 3) (15827, 10)
Shape of X_test, y_test: (4397, 110, 110, 3) (4397, 10)
Shape of X_val, y_val: (1759, 110, 110, 3) (1759, 10)


In [ ]:
import tensorflow as tf
# Define the model architecture (base model)
MainInput = Input(shape=target_shape)
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=target_shape, pooling='avg')(MainInput)

# Add a few-shot learning head on top of the base model
num_classes_per_task = 10  # Number of classes in each few-shot task
few_shot_head = Dense(num_classes_per_task, activation='softmax')(base_model)

# Define the MAML model (outer model)
maml_model = Model(inputs=MainInput, outputs=few_shot_head)

# Define the MAML optimizer and loss function
maml_optimizer = Adam(learning_rate=0.001, epsilon=0.1)

def categorical_crossentropy_loss(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred))

# Perform MAML meta-training
num_tasks = 20  # Number of few-shot tasks for meta-training
num_inner_epochs = 2  # Number of epochs for inner loop training
meta_batch_size = 4  # Number of tasks in each meta-batch

for task in range(num_tasks):
    meta_batch_X = []
    meta_batch_y = []
    
    for _ in range(meta_batch_size):
        # Generate a random few-shot task (support and query sets)
        X_support, X_query, y_support, y_query = train_test_split(X_train, y_train, test_size=num_classes_per_task)
        
        # Perform inner loop training (adaptation to the support set)
        inner_input = Input(shape=target_shape)
        inner_base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=target_shape, pooling='avg')(inner_input)
        inner_few_shot_head = Dense(num_classes_per_task, activation='softmax')(inner_base_model)
        inner_model = Model(inputs=inner_input, outputs=inner_few_shot_head)

        inner_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
        inner_model.fit(X_support, y_support, epochs=num_inner_epochs, batch_size=4, verbose=0)

        # Calculate the adapted model's predictions on the query set
        y_pred_query = inner_model.predict(X_query)
    
    # Stack the meta-batch data and labels
    meta_batch_X = np.vstack(meta_batch_X)
    meta_batch_y = np.vstack(meta_batch_y)
    
    # Perform the meta-update (gradient descent on the meta-parameters)
    with tf.GradientTape() as tape:
        y_pred_meta_batch = maml_model(meta_batch_X)
        loss = categorical_crossentropy_loss(meta_batch_y, y_pred_meta_batch)
    gradients = tape.gradient(loss, maml_model.trainable_variables)
    maml_optimizer.apply_gradients(zip(gradients, maml_model.trainable_variables))

74836368/74836368 [==============================] - 1s 0us/step


In [ ]:
# Evaluate the MAML model on the test set
test_loss, test_accuracy = maml_model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)